In [ ]:
library(tidyverse)
library(factoextra)
source("scripts/r/baypass_utils.R")
source("scripts/r/functions_objects.R")

## Plot population structure of random run

- Plot SVD and PCA, colored by habitat or continent

In [ ]:
omega_mat <- suppressMessages(read_table(snakemake@input[["omega_mat"]], col_names = FALSE))

In [ ]:
samples <- read_delim(snakemake@config[["samples"]], delim='\t') %>% 
    dplyr::select(continent, site, city, range) %>% 
    rename("habitat" = "site") %>% 
    distinct()
head(samples)

In [ ]:
omega_pca <- prcomp(omega_mat)

In [ ]:
pc1_var <- round(get_eigenvalue(omega_pca)["Dim.1", "variance.percent"], 2)
pc2_var <- round(get_eigenvalue(omega_pca)["Dim.2", "variance.percent"], 2)

pca_plot <- get_pca_ind(omega_pca)$coord %>% 
    bind_cols(., samples) %>% 
    ggplot(aes(x = Dim.1, y = Dim.2)) +
        geom_line(aes(group = city)) +
        geom_point(aes(color = continent, shape = habitat), size = 5) +
        scale_color_manual(values = c("#f2cd00", "#ef6a00", "#cd001a", "#538d22", "#1961ae", "#61007d")) +
        xlab(paste0("PC1 (", pc1_var, "%)")) + ylab(paste0("PC2 (", pc2_var, "%)")) +
        my_theme
pca_plot
ggsave(filename = snakemake@output[["pca"]], plot = pca_plot, 
       height = 8, width = 8, device = "pdf", dpi = 600, units = "in")